In [20]:
import requests
import json
import pandas as pd

In [ ]:
# URL criada pelo Query Builder: https://servicodados.ibge.gov.br/api/docs/agregados?versao=3#api-bq
url = "https://servicodados.ibge.gov.br/api/v3/agregados/1613/periodos/2019|2020|2021|2022|2023/variaveis/2313|216|214|112|215?localidades=N8[3101,3102,3103,3104,3105,3106,3107,3108,3109,3110,3111,3112]&classificacao=82[2723,31619,31620]"

In [10]:
req = requests.get(url)
result = req.json()

In [ ]:
with open('results.json', 'w', encoding='utf-8') as file:
    json.dump(result, file, ensure_ascii=False, indent=4)

In [23]:
df_result = pd.DataFrame(columns=[
    "Mesorregião",
    "Produto",
    "Ano",
    "Área destinada à colheita (Hectares)",
    "Área colhida (Hectares)",
    "Quantidade produzida (Toneladas)",
    "Rendimento médio da produção (Quilogramas por Hectare)",
    "Valor da produção (Mil Reais)"
])

In [24]:
anos = [2019,2020,2021,2022,2023]
produtos = [
    "Café (em grão) Total",
    "Café (em grão) Arábica",
    "Café (em grão) Canephora"
]
variaveis = [
    "Área destinada à colheita (Hectares)",
    "Área colhida (Hectares)",
    "Quantidade produzida (Toneladas)",
    "Rendimento médio da produção (Quilogramas por Hectare)",
    "Valor da produção (Mil Reais)"
]

In [26]:
dados = []
mesorregioes = []
for var in result:
    nome_variavel = f"{var['variavel']} ({var["unidade"]})"
    for res in var["resultados"]:
        nome_produto = list(res["classificacoes"][0]["categoria"].values())[0]
        for ser in res["series"]:
            nome_mesorregiao = ser["localidade"]["nome"]
            if nome_mesorregiao not in mesorregioes:
                mesorregioes.append(nome_mesorregiao)
            for ano in anos:
                dados.append({
                    "Mesorregião": nome_mesorregiao,
                    "Produto": nome_produto,
                    "Ano": ano,
                    nome_variavel: ser["serie"][f"{ano}"]
                })

In [29]:
dados

[{'Mesorregião': 'Noroeste de Minas (MG)',
  'Produto': 'Café (em grão) Total',
  'Ano': 2019,
  'Área destinada à colheita (Hectares)': '16255'},
 {'Mesorregião': 'Noroeste de Minas (MG)',
  'Produto': 'Café (em grão) Total',
  'Ano': 2020,
  'Área destinada à colheita (Hectares)': '17744'},
 {'Mesorregião': 'Noroeste de Minas (MG)',
  'Produto': 'Café (em grão) Total',
  'Ano': 2021,
  'Área destinada à colheita (Hectares)': '19008'},
 {'Mesorregião': 'Noroeste de Minas (MG)',
  'Produto': 'Café (em grão) Total',
  'Ano': 2022,
  'Área destinada à colheita (Hectares)': '19051'},
 {'Mesorregião': 'Noroeste de Minas (MG)',
  'Produto': 'Café (em grão) Total',
  'Ano': 2023,
  'Área destinada à colheita (Hectares)': '19960'},
 {'Mesorregião': 'Norte de Minas (MG)',
  'Produto': 'Café (em grão) Total',
  'Ano': 2019,
  'Área destinada à colheita (Hectares)': '7793'},
 {'Mesorregião': 'Norte de Minas (MG)',
  'Produto': 'Café (em grão) Total',
  'Ano': 2020,
  'Área destinada à colheita (

In [27]:
i = 0
for mesorregiao in mesorregioes:
    for produto in produtos:
        for ano in anos:
            df_result.loc[i, "Mesorregião"] = mesorregiao
            df_result.loc[i, "Produto"] = produto
            df_result.loc[i, "Ano"] = ano
            
            for variavel in variaveis:
                for dado in dados:
                    if dado["Mesorregião"] == mesorregiao and dado["Produto"] == produto and dado["Ano"] == ano and variavel in dado:
                        df_result.loc[i, variavel] = dado[variavel]
                        break
            
            i += 1

In [28]:
df_result

,Mesorregião,Produto,Ano,Área destinada à colheita (Hectares),Área colhida (Hectares),Quantidade produzida (Toneladas),Rendimento médio da produção (Quilogramas por Hectare),Valor da produção (Mil Reais)
0,Noroeste de Minas (MG),Café (em grão) Total,2019,16255,16255,37969,2336,254593
1,Noroeste de Minas (MG),Café (em grão) Total,2020,17744,17744,43805,2469,356546
2,Noroeste de Minas (MG),Café (em grão) Total,2021,19008,19008,43050,2265,609797
3,Noroeste de Minas (MG),Café (em grão) Total,2022,19051,19051,44758,2349,928473
4,Noroeste de Minas (MG),Café (em grão) Total,2023,19960,19960,51098,2560,712682
...,...,...,...,...,...,...,...,...
175,Zona da Mata (MG),Café (em grão) Canephora,2019,505,505,1068,2115,5065
176,Zona da Mata (MG),Café (em grão) Canephora,2020,522,522,1233,2362,6623
177,Zona da Mata (MG),Café (em grão) Canephora,2021,504,504,1069,2121,8679
178,Zona da Mata (MG),Café (em grão) Canephora,2022,488,488,1340,2746,15327


In [31]:
df_result.to_excel("dados_ibge/Indicadores IBGE por Mesorregião - 2019 a 2023.xlsx", index=False)